In [1]:
# Import SQL Alchemy
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
import pandas as pd

In [2]:
# Import declarative base and set Base variable
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [3]:
df = pd.read_csv("data.csv")

In [4]:
gb = df.groupby("LocationAbbr").mean().reset_index()

In [5]:
gb.head()

,LocationAbbr,Year,DataValue
0,AK,2013.25,36.919444
1,AL,2013.25,43.458333
2,AR,2013.25,41.886111
3,AZ,2013.25,38.686111
4,CA,2013.25,38.461111


In [6]:
average_dict = {}
for index, row in gb.iterrows():
    average_dict.update({row["LocationAbbr"]:row["DataValue"]})

In [7]:
average_list = []
for index, row in df.iterrows():
    average_list.append(average_dict[row["LocationAbbr"]])

In [8]:
df["Average"] = average_list

In [9]:
question_dict = {
    "Prevalence of diagnosed diabetes among adults aged >= 18 years": "Diabetes",
    "Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes": "Blood Pressure",
    "Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes": "Cholesterol"
}

In [10]:
question_list = []
for index, row in df.iterrows():
    question_list.append(question_dict[row["Question"]])

In [11]:
df["AbbrQuestion"] = question_list

In [12]:
df.head()

,Year,LocationAbbr,LocationDesc,Question,DataValue,Gender,Average,AbbrQuestion
0,2013,AL,Alabama,Prevalence of high blood pressure among adults...,79.4,Female,43.458333,Blood Pressure
1,2015,AL,Alabama,Prevalence of high blood pressure among adults...,78.6,Female,43.458333,Blood Pressure
2,2011,AL,Alabama,Prevalence of high blood pressure among adults...,78.1,Female,43.458333,Blood Pressure
3,2011,AL,Alabama,Prevalence of high blood pressure among adults...,77.8,Overall,43.458333,Blood Pressure
4,2013,AL,Alabama,Prevalence of high blood pressure among adults...,77.6,Overall,43.458333,Blood Pressure


In [13]:
class Data(Base):
    __tablename__ = 'data'
    id = Column(Integer, primary_key=True)
    Year = Column(Integer)
    LocationAbbr = Column(String(255))
    LocationDesc = Column(String(255))
    Question = Column(String(255))
    DataValue = Column(Float)
    Gender = Column(String(255))
    Average = Column(Integer)
    AbbrQuestion = Column(String(255))

In [14]:
engine = create_engine("sqlite:///diabetes.sqlite")
conn = engine.connect()

In [15]:
Base.metadata.create_all(conn)

In [16]:
session = Session(bind=engine)

In [17]:
x = 1

In [18]:
for index, row in df.iterrows():
    data = Data(
        id = x,
        Year=row["Year"],
        LocationAbbr = row["LocationAbbr"],
        LocationDesc = row["LocationDesc"],
        Question = row["Question"],
        DataValue = row["DataValue"],
        Gender = row["Gender"],
        Average = row["Average"],
        AbbrQuestion = row["AbbrQuestion"])
    session.add(data)
    session.commit()
    x +=1

In [19]:
df = pd.read_sql_query("SELECT * FROM data",conn)

In [20]:
df

,id,Year,LocationAbbr,LocationDesc,Question,DataValue,Gender,Average,AbbrQuestion
0,1,2013,AL,Alabama,Prevalence of high blood pressure among adults...,79.4,Female,43.458333,Blood Pressure
1,2,2015,AL,Alabama,Prevalence of high blood pressure among adults...,78.6,Female,43.458333,Blood Pressure
2,3,2011,AL,Alabama,Prevalence of high blood pressure among adults...,78.1,Female,43.458333,Blood Pressure
3,4,2011,AL,Alabama,Prevalence of high blood pressure among adults...,77.8,Overall,43.458333,Blood Pressure
4,5,2013,AL,Alabama,Prevalence of high blood pressure among adults...,77.6,Overall,43.458333,Blood Pressure
5,6,2011,AL,Alabama,Prevalence of high blood pressure among adults...,77.5,Male,43.458333,Blood Pressure
6,7,2015,AL,Alabama,Prevalence of high blood pressure among adults...,77.3,Overall,43.458333,Blood Pressure
7,8,2015,AL,Alabama,Prevalence of high blood pressure among adults...,75.8,Male,43.458333,Blood Pressure
8,9,2013,AL,Alabama,Prevalence of high blood pressure among adults...,75.6,Male,43.458333,Blood Pressure
9,10,2011,AL,Alabama,Prevalence of high cholesterol among adults ag...,75.3,Male,43.458333,Cholesterol
